This FINAL homework recaps the Spark ML library:

0) Download the "Rain in Australia" dataset from Kaggle (it is also attached to this assigbnment): https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

1) In a Jupyter Notebook, write a SparkML script that uses a Decision Tree Classifier to predict the RainTomorrow target varible

2) Split the data 80/20 train/test, using a seed of 12345

3) Use transformers to remove unnecessary columns (use your best judgement) and convert categorical variables into one-hot encoded variables

4) Use a parameter grid to determine the best parameters for:
impurity - gini, entropy
maxBins - 5, 10, 15
minInfoGain - 0.0, 0.2, 0.4
maxDepth - 3, 5, 7

5) Cross-validate with 4 folds

6) Use a pipeline to encapsulate all steps

7) Print the parameters from the best model selected

8) Calculate and print the Area under ROC Curve and Area under Precision-Recall Curve scores for your training and test data sets (these are built-in metrics, you do not need to calculate anything by hand)

Your script should be clean of all the testing and exploration and should only contain the necessary code to satisfy the above conditions

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 51.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=aa98aaa8287ca5dadb48ea98feed2099d6441ace13d1863c4b4c58b5b1b610e6
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pyspark
#I cnonstantly forget to do this part
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [38]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import *
import pandas as pd

mySchema = StructType([
      StructField("Date",StringType(),True),
      StructField("Location",StringType(),True),
      StructField("MinTemp",FloatType(),True),
      StructField("MaxTemp",FloatType(),True),
      StructField("Rainfall",FloatType(),True),
      StructField("Evaporation",FloatType(),True),
      StructField("Sunshine",FloatType(),True),
      StructField("WindGustDir",StringType(),True),
      StructField("WindGustSpeed",IntegerType(),True),
      StructField("WindDir9am",StringType(),True),
      StructField("WindDir3pm",StringType(),True),
      StructField("WindSpeed9am",IntegerType(),True),
      StructField("WindSpeed3pm",IntegerType(),True),
      StructField("Humidity9am",IntegerType(),True),
      StructField("Humidity3pm",IntegerType(),True),
      StructField("Pressure9am",FloatType(),True),
      StructField("Pressure3pm",FloatType(),True),
      StructField("Cloud9am",IntegerType(),True),
      StructField("Cloud3pm",IntegerType(),True),
      StructField("Temp9am",FloatType(),True),
      StructField("Temp3pm",FloatType(),True),
      StructField("RainToday",StringType(),True),
      StructField("RainTomorrow",StringType(),True)
])
data = spark.read.csv("weatherAUS.csv", header=True, nullValue= 'NA', schema=mySchema)


In [39]:
data.show(10)

+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|2008-12-01|  Albury|   13.4|   22.9|     0.6|       null|    null|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       8|    null|   16.9|   21.8|       No|          No|
|2008-12-02|

In [40]:
data.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- MinTemp: float (nullable = true)
 |-- MaxTemp: float (nullable = true)
 |-- Rainfall: float (nullable = true)
 |-- Evaporation: float (nullable = true)
 |-- Sunshine: float (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: integer (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: integer (nullable = true)
 |-- WindSpeed3pm: integer (nullable = true)
 |-- Humidity9am: integer (nullable = true)
 |-- Humidity3pm: integer (nullable = true)
 |-- Pressure9am: float (nullable = true)
 |-- Pressure3pm: float (nullable = true)
 |-- Cloud9am: integer (nullable = true)
 |-- Cloud3pm: integer (nullable = true)
 |-- Temp9am: float (nullable = true)
 |-- Temp3pm: float (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



Sites: 

Lecture slides:
https://docs.google.com/presentation/d/1rM7dnkVBq11PivNYVVFhDjCswybuey2dh47l-K9JUqI/edit#slide=id.p13

That one ML descision tree:
https://spark.apache.org/docs/1.5.2/ml-decision-tree.html

VectorIndexer vs OHE
https://stackoverflow.com/questions/63881128/vectorindexer-or-onehotencoder-for-categorical-variables

Indexor overflow: 
https://stackoverflow.com/questions/56585434/pyspark-pipeline-error-when-using-indexer-and-encoder

data in an excell spreadsheet: 
https://docs.google.com/spreadsheets/d/1P49rE8ZJXb3n4KBv6p5Sm-MPdN5JTJcNfWh5YiXky2o/edit#gid=1304608677

Remember how to read your schema: 
https://predictivehacks.com/?all-tips=define-schema-and-load-data-in-pyspark

Index mult columns:
https://stackoverflow.com/questions/36942233/apply-stringindexer-to-several-columns-in-a-pyspark-dataframe

dfColumns = ["Date", "Location", "MinTemp", "MaxTemp", "Rainfall", "Evaporation",
            "Sunshine", "WindGustDir", "WindGustSpeed", "WindDir9am", "WindDir3pm",
            "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", 
            "Pressure9am", "Pressure3pm", "Cloud9am", "Cloud3pm", "Temp9am",
            "Temp3pm", "RainToday"]

In [48]:
import numpy as np
labelIndexer = StringIndexer(inputCol="RainTomorrow", outputCol="RainLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

categorical_cols = ["Location","WindGustDir", "WindDir9am", "WindDir3pm", "RainToday"] #excluding target column RainTomorrow and Date
numerical_cols = ["MinTemp", "MaxTemp", "Rainfall", "Evaporation", "Sunshine", "WindGustSpeed", "WindSpeed9am", "WindSpeed3pm", "Humidity9am", "Humidity3pm", "Pressure9am", "Pressure3pm", "Cloud9am", "Cloud3pm", "Temp9am", "Temp3pm"]

#dfColumnsVector = np.array(numerical_cols)
#
featureIndexer = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) for column in list(set(data.columns[:-1])-set(['date'])) ]

encoder = OneHotEncoder(inputCols= categorical_cols,
                        outputCols=[cols + "_classVec" for cols in categorical_cols])

featureAssembler = VectorAssembler(inputCols=[cols + "_classVec" for cols in categorical_cols] + numerical_cols, outputCol="assembledFeatures")

(trainingData, testData) = data.randomSplit([0.8, 0.2], seed=12345)
dt = DecisionTreeClassifier(labelCol="RainLabel",
featuresCol="assembledFeatures")
pipeline = Pipeline(stages=[labelIndexer, encoder, featureAssembler, dt])
model = pipeline.fit(trainingData)

IllegalArgumentException: ignored